In [12]:
#!pip install eunjeon #은전 프로젝트 패키지, Mecab 사용 위함.

#밑에 없는 패키지들은
#!pip install 패키지명
#으로 설치하면 됩니다.

In [3]:
import os, re, csv
import pandas as pd
from eunjeon import Mecab
from tqdm import tqdm
import gensim
#import soynlp
mecab = Mecab()

eta_df = pd.read_csv(r"질문.csv", encoding='ANSI')

eta_df.shape

(3024, 6)

# dataframe에서 댓글, 내용 리스트로 가져오기

In [4]:
comments = eta_df['댓글'].tolist()

In [5]:
content = eta_df['내용'].tolist()

# 전처리 (정규화, 불용어 제거)-신우 코드 참조

In [56]:
file = open('stop_words.txt','r', encoding = 'utf8')
lines = file.readlines()
file.close()
stop_words = lines[0].split()

#print(stop_words)

In [57]:
norm_content = []
def text_normalization(content):
    result = []
    for text in content:
        punct = re.compile(r'[\~\'\"\.\,\?\!\:\;\)\(\+]+') 
        norm_text = re.sub(punct, "", text)
        norm_text = re.sub(r'안녕(\s)*(하세요)?', '', norm_text) #안녕, 안녕하세요, 안녕 하세요...
        norm_text = re.sub(r'(조교)[님]?', "", norm_text) #조교, 조교님
        norm_text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]{2,}', ' ', norm_text) #2개 이상 자모 연속 제거 
        norm_text = re.sub( r"\s{2,}"," ", norm_text) #2개 이상 띄어쓰기 1개로 
        norm_text = re.sub( r"\A\s+",'', norm_text) #전처리로 생성된 맨 앞 띄어쓰기 제거
        result.append(norm_text)
    return result

norm_content = text_normalization(content)

#print(norm_content[:20])

In [58]:
### 명사 추출
noun_content = []
for x in norm_content:
    noun_content.append(mecab.nouns(x))

##불용어 제거
def delete_stopwords(norm_content, test=False):
    noun_content = []
    for x in norm_content:
        noun_content.append(mecab.nouns(x))
    
    ### 추출된 명사들을 묶어 merge_content 에 담는다
    merge_content = []
    for noun in noun_content:
        txt = ' '.join(noun)
        merge_content.append(txt)
    
    ### 불용어 리스트에 없는 단어들만 추가, 명사들로 이뤄진 리스트들
    texts = [[word for word in document.split() if word not in stop_words]
            for document in merge_content]
    if test == True:
        return texts[0]
    return texts

texts = delete_stopwords(norm_content)
#print(texts)


###texts 합쳐서 하나의 코퍼스로
# n_merge_content = []

# for noun in texts:
#     txt = ' '.join(noun)
#     n_merge_content.append(txt)




# 구축한 corpus로 doc2vec 모델 만들기

In [60]:
#corpus 읽어서 태그달기(태그는 임의의 태그, 입력된 순서대로 달림)
def read_corpus(noun_content):
    for i, content in tqdm(enumerate(noun_content)):
        yield gensim.models.doc2vec.TaggedDocument(content, [i])

corpus = list(read_corpus(noun_content))
#모델 학습, doc2vec
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40) #모델 생성
model.build_vocab(corpus) #모델 훈련 위한 vocab 생성
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs) #모델 훈련

3024it [00:00, 758133.61it/s]


# 유사한 상위 10개 질문들과 그 답변들

In [61]:
#main
query = input('궁금하신 사항을 입력해 주세요: ')
query = [query] #전처리에 인수 값으로 리스트 형식으로 넣어줘야 하기 때문에 문자열을 포함한 리스트 형식으로 변경

##전처리 진행, 전처리한 문장을 학습한 doc2vec모델에 적용해 vectorize
norm_content = text_normalization(query)
preprocessed = delete_stopwords(norm_content, test = True)
vector = model.infer_vector(preprocessed) #학습된 모델로 벡터값 유추

##가장 유사한 vector를 갖는 질문들 모아모아
sims = model.docvecs.most_similar([vector], topn=len(model.docvecs))

##유사도 상위 10개 tag들
sims_tags = list(map(lambda x: x[0], sims[:10])) 


print('\n\n')
for i, tag in enumerate(sims_tags):
    print(i+1, ')', '유사도: ', sims[i][1])
    print('------질문------')
    print(content[tag])
    print('\n')
    
    print('------답변------')
    comment_devided = comments[tag].split('[')[1].split(']')[0].split("\', \'")
    for j, comment in enumerate(comment_devided):
        print(j+1,')', comment)
    print('\n\n\n\n')

궁금하신 사항을 입력해 주세요: 안녕하세요 조교님! 제가 4-1까지 졸업 필요 학점을 다 채웠는데, 4-2 때 등록금만 내고 수업을 안들어도 되나요?



1 ) 유사도:  0.5782312750816345
------질문------
안녕하세요 조교님! 조기졸업 대상자가 아닌데 4-1에 필수이수학점을 모두 마친 상태라 4-2에 수업을 들을 필요가 없는데도 4-2 등록금을 내야하나요? 그리고 수업은 안들어도 되는지 아니면 하나라도 수강신청해서 들어야하는지도 궁금합니다!


------답변------
1 ) '조기졸업자를 제외하면 8학기까지는 무조건 등록을 해야 졸업이 가능합니다 또한 최소 1학점 이상 수강신청을 해야 등록이 가능하니 8학기에도 한 과목 이상은 들으셔야 합니다'





2 ) 유사도:  0.5360193252563477
------질문------
조교님!! 8학기까지 다들었는데 졸업학점에서 1학점정도가 부족하면 계절학기로 충당하고 졸업할 수 있나요??


------답변------
1 ) '계절학기는 무조건 정규학기 학점으로 산입되어야 합니다 즉 8학기 직후 바로 그 다음 계절을 듣는 경우에만 가능합니다 (8학기까지 마치고 휴학을 했다가 계절만 듣고 졸업하는 것은 불가)
2 ) 답변 주셔서 감사합니다 조교님??'





3 ) 유사도:  0.5239826440811157
------질문------
조교님 안녕하세요만약 4-2까지 학점은 다 채웠는데추가학기에 재수강을 하고 싶으면4-2겨울계절 듣고, 5-1재수강듣고, 여름계절 듣고 졸업하는 것도 가능한가요?


------답변------
1 ) '계절도 전부 재수강으로 생각한 것입니다
2 ) 졸업학점을 다 채우면 그 때부턴 수료생 신분이 되어서 추가학기 수강이 불가능합니다 전공이든 필수과목이든 졸업에 필요한 학점을 1학점이라도 남겨둬야 추가학기 수강이 가능합니다'





4 ) 유사도:  0.5043133497238159
------질문------
조교님 막학기 8학기이면 8학점들어두 등록금 안